### Importação dos dados



In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns

url = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_1.csv"
url2 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_2.csv"
url3 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_3.csv"
url4 = "https://raw.githubusercontent.com/alura-es-cursos/challenge1-data-science/refs/heads/main/base-de-dados-challenge-1/loja_4.csv"

loja = pd.read_csv(url)
loja2 = pd.read_csv(url2)
loja3 = pd.read_csv(url3)
loja4 = pd.read_csv(url4)

loja.head()

In [ ]:
print(f'loja1: {loja.shape[0]} \nloja2: {loja2.shape[0]} \nloja3: {loja3.shape[0]} \nloja54: {loja4.shape[0]}')
print(f'Colunas: {list(loja.columns)}')

In [ ]:
loja['loja'] = 1
loja2['loja'] = 2
loja3['loja'] = 3
loja4['loja'] = 4           

df = pd.concat([loja, loja2, loja3, loja4], ignore_index=True)
df.head()

In [ ]:
# identificando os tipos de dados e dados nulos

df.info()

In [ ]:
df.describe(include='all')

# 1. Analise Faturamento

In [ ]:
df_group_fat = df.groupby('loja')['Preço'].sum().sort_values(ascending=False).reset_index()
df_group_fat.plot(x='loja', y='Preço', kind='bar', figsize=(7, 3), title='Total de vendas por loja', legend=False)  # Removendo a legenda

plt.ticklabel_format(style='plain', axis='y')  
plt.gca().get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x:,.0f}'))  
plt.xticks(rotation=45)  

plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(y=df_group_fat['Preço'])

plt.ticklabel_format(style='plain', axis='y') 
plt.gca().get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x:,.0f}')) 
plt.title('Boxplot Faturamento lojas')
plt.ylabel('Faturamento')

plt.show()

In [ ]:
df_fat_loja = df.copy()
df_fat_loja['Data da Compra'] = pd.to_datetime(df_fat_loja['Data da Compra'], format='%d/%m/%Y')
df_fat_loja['Mes_Ano'] = df_fat_loja['Data da Compra'].dt.to_period('M') 
df_fat_loja['Ano'] = df_fat_loja['Data da Compra'].dt.to_period('Y') 
df_group_fat_loja = df_fat_loja.groupby(['loja','Mes_Ano'])['Preço'].sum().reset_index()

plt.figure(figsize=(20, 4))

colors = cm.tab10(np.linspace(0, 1, len(df_group_fat_loja['loja'].unique())))  # Gera cores únicas para cada loja

for idx, i in enumerate(df_group_fat_loja['loja'].unique()):

    df_aux = df_group_fat_loja[df_group_fat_loja['loja'] == i].copy()
    df_aux = df_aux.sort_values('Mes_Ano')
    df_aux.set_index('Mes_Ano', inplace=True)
    df_aux['CUMSUM'] = df_aux['Preço'].cumsum()

    plt.plot(df_aux.index.astype(str), df_aux['CUMSUM'], label=f'Loja {i}', color=colors[idx])  # Formata o eixo x para exibir Mês-Ano

plt.title('Faturamento Acumulado por Loja ao longo do tempo')
plt.xlabel('Mes_Ano')
plt.ylabel('Faturamento')
plt.legend()
plt.ticklabel_format(style='plain', axis='y')  
plt.gca().get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x:,.0f}')) 
plt.xticks(rotation=45)  
plt.show()


"""
Gráfico de barras agrupado por ano
"""
df_group_fat_loja = df_fat_loja.groupby(['loja','Ano'])['Preço'].sum().reset_index()
df_pivot = df_group_fat_loja.pivot(index='Ano', columns='loja', values='Preço')

bar_width = 0.2

plt.figure(figsize=(10, 4))
anos = df_pivot.index.astype(str)
x = np.arange(len(anos))  # Posições no eixo x


for idx, loja in enumerate(df_pivot.columns):
    plt.bar(x + idx * bar_width, df_pivot[loja], width=bar_width, label=f'Loja {loja}',color=colors[idx])

plt.xticks(x + bar_width * (len(df_pivot.columns) / 2), anos)  # Ajusta posição dos rótulos
plt.title('Faturamento por Loja Agrupado por Ano')
plt.xlabel('Ano')
plt.ylabel('Faturamento')
plt.legend()
plt.ticklabel_format(style='plain', axis='y')  # Evita notação científica
plt.show()

# 2. Vendas por Categoria


In [ ]:
print(f"Categorias:' {len(df['Categoria do Produto'].unique())} \n{list(df['Categoria do Produto'].unique())}")


print('\nContagem de categorias por loja')
df_category_count = df.groupby('loja')['Categoria do Produto'].nunique().reset_index()
df_category_count = df_category_count.sort_values('loja')

print(df_category_count)

In [ ]:
df_group_cat = df.groupby(['loja', 'Categoria do Produto']).agg(
    valor_total=('Preço', 'sum'),
    quantidade=('Preço', 'count')
).reset_index()


"""
GRAFICO DE PIZZA
"""

# Definir quantidade de lojas
num_lojas = len(df_group_cat['loja'].unique())

# Criar a figura com subplots (definindo linhas e colunas automaticamente)
fig, axes = plt.subplots(1, num_lojas, figsize=(6 * num_lojas, 6))

# Garantir que 'axes' seja sempre um array, mesmo quando há apenas uma loja
if num_lojas == 1:
    axes = [axes]

# Iterar sobre cada loja e criar o gráfico de pizza correspondente
for idx, loja in enumerate(df_group_cat['loja'].unique()):
    df_aux = df_group_cat[df_group_cat['loja'] == loja].copy()
    df_aux = df_aux.sort_values('valor_total', ascending=False)
    
    axes[idx].pie(df_aux['valor_total'], labels=df_aux['Categoria do Produto'], autopct='%1.1f%%', 
                  colors=plt.cm.viridis(np.linspace(0, 1, len(df_aux))))
    axes[idx].set_title(f'Faturamento por Categoria - Loja {loja}')

# Ajustar layout para melhor visualização
plt.tight_layout()
plt.show()

""" 
GRAFICO DE PIZZA COM QUANTIDADE"""

# Criar a figura com subplots (uma linha, várias colunas)
fig, axes = plt.subplots(1, num_lojas, figsize=(6 * num_lojas, 6))

# Garantir que 'axes' seja sempre um array, mesmo quando há apenas uma loja
if num_lojas == 1:
    axes = [axes]

# Iterar sobre cada loja e criar o gráfico de pizza correspondente
for idx, loja in enumerate(df_group_cat['loja'].unique()):
    df_aux = df_group_cat[df_group_cat['loja'] == loja].copy()
    df_aux = df_aux.sort_values('quantidade', ascending=False)
    
    # Criar gráfico de pizza com rótulos de quantidades
    axes[idx].pie(df_aux['quantidade'], labels=df_aux['Categoria do Produto'], 
                  autopct=lambda p: f'{int(p * df_aux["quantidade"].sum() / 100)}', 
                  colors=plt.cm.viridis(np.linspace(0, 1, len(df_aux))))
    axes[idx].set_title(f'Quantidade de itens por Categoria - Loja {loja}')

# Ajustar layout para melhor visualização
plt.tight_layout()
plt.show()

"""
GRAFICO DE BARRAS EMPILHADAS
"""

# Transformar os valores em percentuais
df_group_cat['percentual'] = df_group_cat.groupby('loja')['valor_total'].transform(lambda x: x / x.sum() * 100)

# Criar tabela pivô para barras empilhadas
df_pivot = df_group_cat.pivot(index='loja', columns='Categoria do Produto', values='percentual')

# Configurar posições e largura das barras
bar_width = 0.8
x = np.arange(len(df_pivot.index))  # Posições no eixo x

plt.figure(figsize=(14, 6))
bottom = np.zeros(len(df_pivot.index))  # Inicializador para empilhamento

# Iterar sobre as categorias e empilhar as barras
for categoria in df_pivot.columns:
    bars = plt.bar(x, df_pivot[categoria], width=bar_width, label=categoria, bottom=bottom)
    
    # Adicionar rótulos diretamente nas barras
    for bar in bars:
        height = bar.get_height()
        if height > 2:  # Exibir apenas valores significativos
            plt.text(bar.get_x() + bar.get_width() / 2, bar.get_y() + height / 2, f'{height:.1f}%', ha='center', va='center', fontsize=10, color='white')

    bottom += df_pivot[categoria].fillna(0)  # Atualizar posição para empilhamento

# Ajustar rótulos e título
plt.xticks(x, df_pivot.index, rotation=45)
plt.title('Distribuição Percentual do Faturamento por Loja')
plt.xlabel('Loja')
plt.ylabel('Percentual do Faturamento (%)')
plt.gca().get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, pos: f'{x:.0f}%'))  # Formatar eixo y como percentual

# Posicionar a legenda fora do gráfico (lado direito)
plt.legend(title='Categoria do Produto', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()

# 3. Média de Avaliação das Lojas

In [ ]:
df_cat_aux = df.groupby('Categoria do Produto').agg(valor_total=('Preço', 'sum'), quantidade=('Produto', 'count')).sort_values('valor_total', ascending=False).reset_index()
df_cat_aux['percentual'] = df_cat_aux['valor_total'] / df_cat_aux['valor_total'].sum() * 100
df_cat_aux['cumsum'] = df_cat_aux['percentual'].cumsum()
df_cat_aux['cumsum'] = df_cat_aux['cumsum'].round(2)
CATEGORIAS_RELEVANTES = df_cat_aux[df_cat_aux['cumsum']<90]['Categoria do Produto'].unique()

In [ ]:
df.head(5)

In [ ]:
df_aux = df.copy()
df_group_a = df_aux.groupby('loja')['Avaliação da compra'].mean().reset_index()

ax = df_group_a.plot(x='loja', y='Avaliação da compra', kind='bar', figsize=(7, 3), title='Média de Avaliação por Loja (TOTAL)', legend=False)

plt.ylim(0, 5)  # Define o limite do eixo y
plt.ticklabel_format(style='plain', axis='y')
plt.gca().get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x:,.0f}'))
plt.xticks(rotation=45)

# Adicionar rótulos sobre as barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', 
                (p.get_x() + p.get_width() / 2, min(p.get_height(), 5)),  # Garante que os rótulos fiquem dentro do limite
                ha='center', va='bottom', fontsize=8, color='black')

plt.show()


df_aux = df.copy()
df_aux = df_aux[df_aux['Categoria do Produto'].isin(CATEGORIAS_RELEVANTES)]
df_group_a = df_aux.groupby('loja')['Avaliação da compra'].mean().reset_index()

ax = df_group_a.plot(x='loja', y='Avaliação da compra', kind='bar', figsize=(7, 3), title='Média de Avaliação por Loja (categorias mais relevantes)', legend=False)

plt.ylim(0, 5)  # Define o limite do eixo y
plt.ticklabel_format(style='plain', axis='y')
plt.gca().get_yaxis().set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{x:,.0f}'))
plt.xticks(rotation=45)

# Adicionar rótulos sobre as barras
for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', 
                (p.get_x() + p.get_width() / 2, min(p.get_height(), 5)),  # Garante que os rótulos fiquem dentro do limite
                ha='center', va='bottom', fontsize=8, color='black')

plt.show()


In [ ]:
plt.figure(figsize=(4, 4))
sns.boxplot(y=df_group_a['Avaliação da compra'])

# Calcular média e mediana
media = df_group_a['Avaliação da compra'].mean()
mediana = df_group_a['Avaliação da compra'].median()

# Adicionar linhas para média e mediana
plt.axhline(media, color='blue', linestyle='--', label=f'Média: {media:.2f}')
plt.axhline(mediana, color='red', linestyle='-', label=f'Mediana: {mediana:.2f}')

plt.title('Boxplot Avaliação da Compra (MÉDIAS) - GERAL')
plt.ylabel('Avaliação')
# plt.legend(loc='upper right')  # Posicionar a legenda
plt.legend(title='Legenda', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.show()


In [ ]:
plt.figure(figsize=(12, 6))  # Ajustar tamanho da figura
sns.boxplot(x='loja', y='Avaliação da compra', data=df)
# sns.boxplot(x='loja', y='Avaliação da compra', data=df, showfliers=False)  # Remover outliers

plt.title('Boxplot das Avaliações por Loja')
plt.xlabel('Loja')
plt.ylabel('Avaliação da compra')
plt.xticks(rotation=45)  # Melhorar legibilidade dos rótulos

plt.show()

In [ ]:
df_a = df.copy()
df_a['Data da Compra'] = pd.to_datetime(df_a['Data da Compra'], format='%d/%m/%Y')
df_a['Mes_Ano'] = df_a['Data da Compra'].dt.to_period('M') 
df_a['Ano'] = df_a['Data da Compra'].dt.to_period('Y') 

# Definir quantidade de lojas
num_lojas = len(df_a['loja'].unique())

# Criar subplots (uma linha com várias colunas)
fig, axes = plt.subplots(1, num_lojas, figsize=(6 * num_lojas, 6), sharey=True)

# Garantir que 'axes' seja sempre uma lista, mesmo se houver apenas uma loja
if num_lojas == 1:
    axes = [axes]

# Gerar boxplots para cada loja
for idx, loja in enumerate(df_a['loja'].unique()):
    df_aux = df_a[df_a['loja'] == loja].sort_values('Ano').copy()
    
    sns.boxplot(x='Ano', y='Avaliação da compra', data=df_aux, ax=axes[idx])  # Remover outliers
    axes[idx].set_title(f'Avaliação por Ano - Loja {loja}')
    axes[idx].set_xlabel('Ano')
    axes[idx].set_ylabel('Avaliação')
    axes[idx].tick_params(axis='x', rotation=45)  # Girar rótulos do eixo x para melhor legibilidade

# Ajustar layout para melhor visualização
plt.tight_layout()
plt.show()


In [ ]:
df_time_ano = df_a.groupby(['Ano', 'loja'])['Avaliação da compra'].mean().reset_index()
df_time_ano['Media_Movel'] = df_time_ano.groupby('loja')['Avaliação da compra'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

df_time_ano['Ano'] = df_time_ano['Ano'].astype(str)  # Converter para string

plt.figure(figsize=(12, 6))

# Iterar sobre as lojas e plotar com as cores definidas
for idx, loja in enumerate(df_time_ano['loja'].unique()):
    df_aux = df_time_ano[df_time_ano['loja'] == loja]
    sns.lineplot(x='Ano', y='Media_Movel', data=df_aux, marker='o', color=colors[idx], label=f'Loja {loja}')

plt.title('Média Móvel das Avaliações das Lojas por Ano', fontsize=16)
plt.xlabel('Ano')
plt.ylabel('Avaliação Média')
plt.xticks(rotation=45)
plt.legend(title='Loja')

plt.show()

In [ ]:

df_time = df_a.groupby(['Mes_Ano', 'loja'])['Avaliação da compra'].mean().reset_index()
df_time['Media_Movel'] = df_time.groupby('loja')['Avaliação da compra'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())
df_time['Mes_Ano'] = df_time['Mes_Ano'].astype(str)  # Converter para string se necessário

plt.figure(figsize=(20, 4))

# Iterar sobre as lojas e plotar com as cores definidas
for idx, loja in enumerate(df_time['loja'].unique()):
    df_aux = df_time[df_time['loja'] == loja]
    sns.lineplot(x='Mes_Ano', y='Media_Movel', data=df_aux, marker='o', color=colors[idx], label=f'Loja {loja}')

plt.title('Média Móvel das Avaliações das Lojas',fontsize=16)
plt.xlabel('Mês-Ano')
plt.ylabel('Avaliação Média')
plt.xticks(rotation=45)
plt.legend(title='Loja')

plt.show()

In [ ]:
# Criar um boxplot separado para cada loja
for loja in df['loja'].unique():
    df_aux = df[df['loja'] == loja].copy()

    plt.figure(figsize=(8, 3))
    sns.boxplot(x='Categoria do Produto', y='Avaliação da compra', data=df_aux, showfliers=False)  # Remover outliers
    
    plt.title(f'Boxplot das Avaliações por Categoria - Loja {loja}')
    plt.xlabel('Categoria do Produto')
    plt.ylabel('Avaliação da compra')
    plt.xticks(rotation=45)  # Melhorar legibilidade dos rótulos
    
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Agrupar os dados para obter a média por loja e categoria
df_media = df.groupby(['loja', 'Categoria do Produto'])['Avaliação da compra'].mean().reset_index()

# Calcular a média geral por categoria
df_media_geral = df.groupby('Categoria do Produto')['Avaliação da compra'].mean().reset_index()

plt.figure(figsize=(12, 6))

# Plotar as médias individuais das lojas por categoria
sns.barplot(x='Categoria do Produto', y='Avaliação da compra', hue='loja', data=df_media, palette=colors)

# Adicionar apenas os marcadores da média geral por categoria
sns.pointplot(x='Categoria do Produto', y='Avaliação da compra', data=df_media_geral, color='black', markers='_', linestyles="", scale=2, label='Média Geral', ci=None)

plt.title('Média de Avaliação por Loja e Categoria (com Marcadores de Média Geral)')
plt.xlabel('Categoria do Produto')
plt.ylabel('Avaliação Média')
plt.xticks(rotation=45)
plt.legend(title='Lojas', bbox_to_anchor=(1.05, 1), loc='upper left')  # Legenda fora do gráfico
plt.ylim(3.5, 4.5)

plt.show()

# 4. Produtos Mais e Menos Vendidos

# 5. Frete Médio por Loja
